#### Setting up Google Palm LLM & API Key

In [224]:
from langchain.llms import GooglePalm

api_key = "Your API Key"

llm = GooglePalm(google_api_key = api_key, temperature = 0.2)

In [8]:
view = llm("what type of changes needed in school result, give answer in 5 lines")
print(view)

**5 Types of Changes Needed in School to Improve Results**

1. **Increased teacher training and support.**
2. **More resources for students and schools.**
3. **Changes to the way students are taught.**
4. **Increased parental involvement.**
5. **A focus on student success.**


### Connect with database and ask some basic questions


In [9]:
from langchain.utilities import SQLDatabase

In [10]:
db_user = "root"
db_password = "Infi07"
db_host = "localhost"
db_name = "school_result"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE marks (
	mark_id INTEGER NOT NULL, 
	student_id VARCHAR(10), 
	subject_code VARCHAR(30), 
	subject_name VARCHAR(100), 
	marks_obtained INTEGER, 
	grade CHAR(1), 
	PRIMARY KEY (mark_id), 
	CONSTRAINT marks_ibfk_1 FOREIGN KEY(student_id) REFERENCES students (student_id), 
	CONSTRAINT marks_ibfk_2 FOREIGN KEY(subject_code) REFERENCES teachers (teacher_id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from marks table:
mark_id	student_id	subject_code	subject_name	marks_obtained	grade
1	SI01	EN	English	82	A
2	SI01	CO	Computer	58	D
3	SI01	HI	History	49	E
*/


CREATE TABLE students (
	student_id VARCHAR(10) NOT NULL, 
	student_name VARCHAR(100), 
	standard ENUM('6th','7th','8th'), 
	gender ENUM('Male','Female','Other'), 
	coaching_class ENUM('Yes','No'), 
	PRIMARY KEY (student_id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from students table:
student_id	student_name	standard	gender	coaching_class
EI01	Piyusha Kada

In [57]:
# create sql database chain, which take two add llm and db
# we can one more para which is varbose for extra information, squery info

from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [95]:
# "Find out which subject Ankita Mundada teaches"

qu1 = db_chain.run("Find out which subject Ankita Mundada teaches?")



> Entering new SQLDatabaseChain chain...
Find out which subject Ankita Mundada teaches?
SQLQuery:SELECT subject_taught FROM teachers WHERE teacher_name = 'Ankita Mundada'
SQLResult: [('Maths',)]
Answer:Maths
> Finished chain.


In [96]:
# "what is percentage of student id SI07"

qu2 = db_chain.run("SELECT s.student_id, s.student_name, ROUND((SUM(m.marks_obtained) / COUNT(m.mark_id)), 2) AS percentage FROM students s JOIN marks m USING(student_id) WHERE s.student_id = 'SI07';")



> Entering new SQLDatabaseChain chain...
SELECT s.student_id, s.student_name, ROUND((SUM(m.marks_obtained) / COUNT(m.mark_id)), 2) AS percentage FROM students s JOIN marks m USING(student_id) WHERE s.student_id = 'SI07';
SQLQuery:SELECT s.student_id, s.student_name, ROUND((SUM(m.marks_obtained) / COUNT(m.mark_id)), 2) AS percentage FROM students s JOIN marks m USING(student_id) WHERE s.student_id = 'SI07';
SQLResult: [('SI07', 'Kabir Thapar', Decimal('49.20'))]
Answer:Kabir Thapar, 49.20
> Finished chain.


In [70]:
#  qu3 find out who got the highest marks in Maths among 8th standard students and how many marks they scored.

qu3 = db_chain.run("find out who got the highest marks in Maths among 8th standard students and how many marks they scored")
print(qu3)



> Entering new SQLDatabaseChain chain...
find out who got the highest marks in Maths among 8th standard students and how many marks they scored
SQLQuery:SELECT student_name, marks_obtained FROM marks JOIN students ON marks.student_id = students.student_id WHERE subject_name = 'Maths' AND standard = '8th' ORDER BY marks_obtained DESC LIMIT 1
SQLResult: [('Nikhil Pandit', 100)]
Answer:Nikhil Pandit scored 100 marks in Maths
> Finished chain.
Nikhil Pandit scored 100 marks in Maths


In [72]:
# Find the average marks in English of 7th standard students

qu4 = db_chain.run("Find the average marks in English of 7th standard students")



> Entering new SQLDatabaseChain chain...
find the average marks in English of 7th standard students
SQLQuery:SELECT AVG(marks_obtained) FROM marks WHERE subject_name = 'English' AND student_id IN (SELECT student_id FROM students WHERE standard = '7th')
SQLResult: [(Decimal('69.3200'),)]
Answer:69.32
> Finished chain.


In [76]:
# What is the average marks per standard for the subject taught by Sohel Khan

qu5 = db_chain.run("find the average marks of students taught by Sohel Khan per standard")



> Entering new SQLDatabaseChain chain...
find the average marks of students taught by Sohel Khan per standard
SQLQuery:SELECT AVG(marks_obtained) AS average_marks, standard FROM marks JOIN students ON marks.student_id = students.student_id JOIN teachers ON marks.subject_code = teachers.teacher_id WHERE teachers.teacher_name = 'Sohel Khan' GROUP BY standard
SQLResult: [(Decimal('71.2800'), '6th'), (Decimal('58.2800'), '7th'), (Decimal('64.7200'), '8th')]
Answer:(71.2800, 6th), (58.2800, 7th), (64.7200, 8th)
> Finished chain.


In [83]:
# "Find out how many students got more than 70% in class 7th"

qu6 = db_chain.run(""" SELECT COUNT(*) AS num_students_above_70_percent
    FROM ( SELECT s.student_id FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.standard = '7th'
    GROUP BY s.student_id HAVING ROUND((SUM(m.marks_obtained) / (COUNT(m.mark_id) * 100)) * 100, 2) > 70) AS students_above_70_percent;""")



> Entering new SQLDatabaseChain chain...
 SELECT COUNT(*) AS num_students_above_70_percent
FROM ( SELECT s.student_id FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.standard = '7th'
    GROUP BY s.student_id HAVING ROUND((SUM(m.marks_obtained) / (COUNT(m.mark_id) * 100)) * 100, 2) > 70) AS students_above_70_percent;
SQLQuery:SELECT COUNT(*) AS num_students_above_70_percent
FROM ( SELECT s.student_id FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.standard = '7th'
    GROUP BY s.student_id HAVING ROUND((SUM(m.marks_obtained) / (COUNT(m.mark_id) * 100)) * 100, 2) > 70) AS students_above_70_percent;
SQLResult: [(7,)]
Answer:7
> Finished chain.


In [159]:
# "Give information of student id SI02"

qu7 = db_chain.run("Give information of student id SI02")



> Entering new SQLDatabaseChain chain...
Give information of student id SI02
SQLQuery:SELECT student_name, standard, gender, coaching_class FROM students WHERE student_id = 'SI02'
SQLResult: [('Ganesh Malhotra', '6th', 'Male', 'Yes')]
Answer:Ganesh Malhotra, 6th, Male, Yes
> Finished chain.


In [90]:
# "find out in how many sunject students get an A grade in class 6th"

qu8 = db_chain.run("How many students gets A grade in class 6th")



> Entering new SQLDatabaseChain chain...
How many students gets A grade in class 6th
SQLQuery:SELECT COUNT(*) FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '6th' AND m.grade = 'A'
SQLResult: [(35,)]
Answer:35
> Finished chain.


In [93]:
# "Find out how many students failed in class 7th"

qu9 = db_chain.run("""SELECT s.standard, COUNT(DISTINCT s.student_id) AS num_failed_students FROM students s
    JOIN marks m ON s.student_id = m.student_id WHERE s.standard IN ('7th')
    AND m.grade = 'F' GROUP BY s.standard; """)



> Entering new SQLDatabaseChain chain...
SELECT s.standard, COUNT(DISTINCT s.student_id) AS num_failed_students FROM students s
JOIN marks m ON s.student_id = m.student_id WHERE s.standard IN ('7th')
AND m.grade = 'F' GROUP BY s.standard; 
SQLQuery:SELECT s.standard, COUNT(DISTINCT s.student_id) AS num_failed_students FROM students s
JOIN marks m ON s.student_id = m.student_id WHERE s.standard IN ('7th')
AND m.grade = 'F' GROUP BY s.standard;
SQLResult: [('7th', 13)]
Answer:13
> Finished chain.


In [94]:
# "Find the subject in which the most students failed and their count for class 6th"

qu10 = db_chain.run("""SELECT m.subject_name, COUNT(*) AS num_failed_students FROM marks m
    JOIN students s ON s.student_id = m.student_id WHERE s.standard = '8th' AND m.grade = 'F'
    GROUP BY m.subject_name ORDER BY num_failed_students DESC LIMIT 1;""")



> Entering new SQLDatabaseChain chain...
SELECT m.subject_name, COUNT(*) AS num_failed_students FROM marks m
JOIN students s ON s.student_id = m.student_id WHERE s.standard = '8th' AND m.grade = 'F'
GROUP BY m.subject_name ORDER BY num_failed_students DESC LIMIT 1;
SQLQuery:SELECT m.subject_name, COUNT(*) AS num_failed_students FROM marks m
JOIN students s ON s.student_id = m.student_id WHERE s.standard = '8th' AND m.grade = 'F'
GROUP BY m.subject_name ORDER BY num_failed_students DESC LIMIT 1;
SQLResult: [('History', 4)]
Answer:History
> Finished chain.


In [97]:
# "Find out the number of male and female students in each class"

qu11 = db_chain.run("Find out the number of male and female students in each class")



> Entering new SQLDatabaseChain chain...
find out the number of male and female students in each class
SQLQuery:SELECT standard, gender, COUNT(*) AS count FROM students GROUP BY standard, gender
SQLResult: [('8th', 'Female', 13), ('8th', 'Male', 12), ('7th', 'Female', 12), ('7th', 'Male', 13), ('6th', 'Female', 15), ('6th', 'Male', 10)]
Answer:8th: Female 13, Male 12; 7th: Female 12, Male 13; 6th: Female 15, Male 10
> Finished chain.


In [99]:
#  "Determine the subjects taught by each teacher"

qu12 = db_chain.run("determine the subjects taught by each teacher name")



> Entering new SQLDatabaseChain chain...
determine the subjects taught by each teacher name
SQLQuery:SELECT teacher_name, subject_taught FROM teachers
SQLResult: [('Maria DeSouza', 'Computer'), ('Rajat Verma', 'English'), ('Sohel Khan', 'History'), ('Ankita Mundada', 'Maths'), ('Sonal Sardesai', 'Science')]
Answer:Maria DeSouza -> Computer, Rajat Verma -> English, Sohel Khan -> History, Ankita Mundada -> Maths, Sonal Sardesai -> Science
> Finished chain.


In [101]:
# calculate the average marks obtained by students in each subject for class 7th

qu13 = db_chain.run("""SELECT subject_name, ROUND(AVG(marks_obtained), 2) AS average_marks FROM marks
    JOIN students ON marks.student_id = students.student_id
    WHERE students.standard = '7th' GROUP BY subject_name;""")



> Entering new SQLDatabaseChain chain...
SELECT subject_name, ROUND(AVG(marks_obtained), 2) AS average_marks FROM marks
    JOIN students ON marks.student_id = students.student_id
    WHERE students.standard = '7th' GROUP BY subject_name;
SQLQuery:SELECT subject_name, ROUND(AVG(marks_obtained), 2) AS average_marks FROM marks
   JOIN students ON marks.student_id = students.student_id
   WHERE students.standard = '7th' GROUP BY subject_name;
SQLResult: [('English', Decimal('69.32')), ('Computer', Decimal('64.60')), ('History', Decimal('58.28')), ('Maths', Decimal('63.84')), ('Science', Decimal('68.80'))]
Answer:English, 69.32
> Finished chain.


In [163]:
# Find out the total number of students standardwise who attend coaching classes

qu14 = db_chain.run(""" SELECT standard, COUNT(*) AS total_students,
SUM(CASE WHEN coaching_class = 'Yes' THEN 1 ELSE 0 END) AS num_students_attending_coaching FROM students GROUP BY standard;""")



> Entering new SQLDatabaseChain chain...
 SELECT standard, COUNT(*) AS total_students,
SUM(CASE WHEN coaching_class = 'Yes' THEN 1 ELSE 0 END) AS num_students_attending_coaching FROM students GROUP BY standard;
SQLQuery:SELECT standard, COUNT(*) AS total_students,
SUM(CASE WHEN coaching_class = 'Yes' THEN 1 ELSE 0 END) AS num_students_attending_coaching FROM students GROUP BY standard;
SQLResult: [('8th', 25, Decimal('13')), ('7th', 25, Decimal('11')), ('6th', 25, Decimal('14'))]
Answer:8th, 25, 13
> Finished chain.


In [115]:
# List the names of students who scored the highest marks in each subject of 8th std

qu15 = db_chain.run("""WITH SubjectTopper AS ( SELECT m.subject_name, s.student_id, s.student_name, m.marks_obtained AS highest_marks,
	ROW_NUMBER() OVER (PARTITION BY m.subject_name ORDER BY m.marks_obtained DESC) AS rr
    FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '8th')
	SELECT subject_name, student_id, student_name, highest_marks FROM SubjectTopper WHERE rr = 1;""")



> Entering new SQLDatabaseChain chain...
WITH SubjectTopper AS ( SELECT m.subject_name, s.student_id, s.student_name, m.marks_obtained AS highest_marks,
	ROW_NUMBER() OVER (PARTITION BY m.subject_name ORDER BY m.marks_obtained DESC) AS rr
    FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '8th')
	SELECT subject_name, student_id, student_name, highest_marks FROM SubjectTopper WHERE rr = 1;
SQLQuery:WITH SubjectTopper AS ( SELECT m.subject_name, s.student_id, s.student_name, m.marks_obtained AS highest_marks,
	ROW_NUMBER() OVER (PARTITION BY m.subject_name ORDER BY m.marks_obtained DESC) AS rr
    FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '8th')
	SELECT subject_name, student_id, student_name, highest_marks FROM SubjectTopper WHERE rr = 1;
SQLResult: [('Computer', 'EI11', 'Omkar Khair', 97), ('English', 'EI11', 'Omkar Khair', 93), ('History', 'EI17', 'Ranjan Kumar', 96), ('Maths', 'EI10', 'Nikhil Pandit', 100), ('Scien

In [125]:
# Determine the number of students who failed in more than one subjects of class 7th.

qu16 = db_chain.run("""SELECT COUNT(*) AS num_students_failed_in_more_than_two_subjects
    FROM ( SELECT m.student_id FROM marks m JOIN students s ON m.student_id = s.student_id
    WHERE s.standard = '7th' AND m.grade = 'F' GROUP BY m.student_id
    HAVING COUNT(*) > 1) AS failed_student""")



> Entering new SQLDatabaseChain chain...
SELECT COUNT(*) AS num_students_failed_in_more_than_two_subjects
FROM ( SELECT m.student_id FROM marks m JOIN students s ON m.student_id = s.student_id
WHERE s.standard = '7th' AND m.grade = 'F' GROUP BY m.student_id
HAVING COUNT(*) > 1) AS failed_student
SQLQuery:SELECT COUNT(*) AS num_students_failed_in_more_than_two_subjects
FROM ( SELECT m.student_id FROM marks m JOIN students s ON m.student_id = s.student_id
WHERE s.standard = '7th' AND m.grade = 'F' GROUP BY m.student_id
HAVING COUNT(*) > 1) AS failed_student
SQLResult: [(4,)]
Answer:4
> Finished chain.


In [130]:
# Calculate the overall pass percentage of students in each class

qu17 = db_chain.run("""SELECT standard, ROUND((COUNT(CASE WHEN grade != 'F' THEN 1 END) / COUNT(*) * 100), 2) AS pass_percentage
    FROM students JOIN marks ON students.student_id = marks.student_id GROUP BY standard ORDER BY standard;""")



> Entering new SQLDatabaseChain chain...
SELECT standard, ROUND((COUNT(CASE WHEN grade != 'F' THEN 1 END) / COUNT(*) * 100), 2) AS pass_percentage
    FROM students JOIN marks ON students.student_id = marks.student_id GROUP BY standard ORDER BY standard;
SQLQuery:SELECT standard, ROUND((COUNT(CASE WHEN grade != 'F' THEN 1 END) / COUNT(*) * 100), 2) AS pass_percentage
   FROM students JOIN marks ON students.student_id = marks.student_id GROUP BY standard ORDER BY standard;
SQLResult: [('6th', Decimal('91.20')), ('7th', Decimal('85.60')), ('8th', Decimal('88.00'))]
Answer:6th, 7th, 8th
> Finished chain.


In [134]:
# Retrieve the name of the student who got an 'A' grade in computer for class 6th

qu18 = db_chain.run("""retrieve the name of the student who got an 'A' grade in computer for class 6th""")



> Entering new SQLDatabaseChain chain...
retrieve the name of the student who got an 'A' grade in computer for class 6th
SQLQuery:SELECT student_name FROM students JOIN marks ON students.student_id = marks.student_id JOIN teachers ON marks.subject_code = teachers.teacher_id WHERE marks.subject_name = 'Computer' AND marks.grade = 'A' AND students.standard = '6th'
SQLResult: [('Jyoti Sharma',), ('Neeraj Rajput',), ('Saurav Minati',), ('Kiran Kirtan',), ('Sarika Devi',), ('Seema Kadam',), ('Avanish Kadam',)]
Answer:Jyoti Sharma
> Finished chain.


In [143]:
# qu19 display the results of students SI01 and SI07 in a comparative form

qu19 = db_chain.run(""" SELECT subject_name,
    MAX(CASE WHEN student_id = 'SI01' THEN marks_obtained END) AS SI01_marks,
    MAX(CASE WHEN student_id = 'SI07' THEN marks_obtained END) AS SI07_marks,
    MAX(CASE WHEN student_id = 'SI01' THEN grade END) AS SI01_grade,
    MAX(CASE WHEN student_id = 'SI07' THEN grade END) AS SI07_grade
    FROM marks WHERE student_id IN ('SI01', 'SI07') GROUP BY subject_name;""")



> Entering new SQLDatabaseChain chain...
 SELECT subject_name,
    MAX(CASE WHEN student_id = 'SI01' THEN marks_obtained END) AS SI01_marks,
    MAX(CASE WHEN student_id = 'SI07' THEN marks_obtained END) AS SI07_marks,
    MAX(CASE WHEN student_id = 'SI01' THEN grade END) AS SI01_grade,
    MAX(CASE WHEN student_id = 'SI07' THEN grade END) AS SI07_grade
    FROM marks WHERE student_id IN ('SI01', 'SI07') GROUP BY subject_name;
SQLQuery:SELECT subject_name,
   MAX(CASE WHEN student_id = 'SI01' THEN marks_obtained END) AS SI01_marks,
   MAX(CASE WHEN student_id = 'SI07' THEN marks_obtained END) AS SI07_marks,
   MAX(CASE WHEN student_id = 'SI01' THEN grade END) AS SI01_grade,
   MAX(CASE WHEN student_id = 'SI07' THEN grade END) AS SI07_grade
   FROM marks WHERE student_id IN ('SI01', 'SI07') GROUP BY subject_name;
SQLResult: [('English', 82, 30, 'A', 'F'), ('Computer', 58, 59, 'D', 'D'), ('History', 49, 56, 'E', 'D'), ('Maths', 35, 48, 'F', 'E'), ('Science', 72, 53, 'B', 'D')]
Answer:E

In [153]:
# find out first, second and third ranker of class 8th

qu20 = db_chain.run(""" SELECT m.student_id, s.student_name,'7th' AS class, SUM(m.marks_obtained) AS total_marks FROM marks m
JOIN students s ON m.student_id = s.student_id WHERE s.standard = '7th'
GROUP BY m.student_id, s.student_name ORDER BY total_marks DESC LIMIT 3;""")



> Entering new SQLDatabaseChain chain...
 SELECT m.student_id, s.student_name,'7th' AS class, SUM(m.marks_obtained) AS total_marks FROM marks m
JOIN students s ON m.student_id = s.student_id WHERE s.standard = '7th'
GROUP BY m.student_id, s.student_name ORDER BY total_marks DESC LIMIT 3;
SQLQuery:SELECT m.student_id, s.student_name,'7th' AS class, SUM(m.marks_obtained) AS total_marks FROM marks m
JOIN students s ON m.student_id = s.student_id WHERE s.standard = '7th'
GROUP BY m.student_id, s.student_name ORDER BY total_marks DESC LIMIT 3;
SQLResult: [('SE06', 'Arjun Kadam', '7th', Decimal('384')), ('SE11', 'Rajan Ray', '7th', Decimal('377')), ('SE07', 'Kritika Trivedi', '7th', Decimal('375'))]
Answer:Arjun Kadam, Rajan Ray, Kritika Trivedi
> Finished chain.


#### Few Shot Learning

We will use few shot learning to fix issues we have seen so far


In [164]:
few_shots = [
    {'Question': "Find out which subject Ankita Mundada teaches",
     'SQLQuery': "SELECT subject_taught FROM teachers WHERE teacher_name = 'Ankita Mundada'",
     'SQLResult': "Result of the SQL query",
     'Answer': qu1},

    {'Question': "Percentage of student id SI07",
     'SQLQuery': "SELECT s.student_id, s.student_name, ROUND((SUM(m.marks_obtained) / COUNT(m.mark_id)), 2) AS percentage FROM students s JOIN marks m USING(student_id) WHERE s.student_id = 'SI07'",
     'SQLResult': "Result of the SQL query",
     'Answer': qu2},

    {'Question': "Find out who got the highest marks in Maths among 8th standard students and how many marks they scored.",
     'SQLQuery': "SELECT s.student_name, m.marks_obtained AS highest_marks FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.standard = '8th' AND m.subject_name = 'Maths' ORDER BY m.marks_obtained DESC LIMIT 1",
     'SQLResult': "Result of the SQL query",
     'Answer': qu3},

    {'Question': "Find the average marks in English of 7th standard students?",
     'SQLQuery': "SELECT ROUND(AVG(marks_obtained), 2) AS average_marks FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '7th' AND m.subject_name = 'English'",
     'SQLResult': "Result of the SQL query",
     'Answer': qu4},
     
    {'Question': "Find the average marks per standard for the subject taught by Sohel Khan",
     'SQLQuery': "SELECT s.standard, ROUND(AVG(m.marks_obtained), 2) AS average_marks FROM marks m JOIN teachers t ON m.subject_code = t.teacher_id JOIN students s ON m.student_id = s.student_id WHERE t.teacher_name = 'Sohel Khan' GROUP BY s.standard",
     'SQLResult': "Result of the SQL query",
     'Answer': qu5},

    {'Question': "Find out how many students got more than 70% in class 7th",
     'SQLQuery': "SELECT COUNT(*) AS num_students_above_70_percent FROM ( SELECT s.student_id FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.standard = '7th' GROUP BY s.student_id HAVING ROUND((SUM(m.marks_obtained) / (COUNT(m.mark_id) * 100)) * 100, 2) > 70 ) AS students_above_70_percent",
     'SQLResult': "Result of the SQL query",
     'Answer': qu6},

    {'Question': "Find out the information of student id SI02",
     'SQLQuery': "SELECT student_name, standard, gender, coaching_class FROM students WHERE student_id = 'SI02'",
     'SQLResult': "Result of the SQL query",
     'Answer': qu7},

    {'Question': "Find out how many students get an A grade in class 6th",
     'SQLQuery': "SELECT COUNT(*) FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '6th' AND m.grade = 'A'",
     'SQLResult': "Result of the SQL query",
     'Answer': qu8},

    {'Question': "Find out how many students failed in class 7th",
     'SQLQuery': "SELECT s.standard, COUNT(DISTINCT s.student_id) AS num_failed_students FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.standard IN ('7th') AND m.grade = 'F' GROUP BY s.standard",
     'SQLResult': "Result of the SQL query",
     'Answer': qu9},

    {'Question': "Find the subject in which the most students failed and their count for class 6th",
     'SQLQuery': "SELECT m.subject_name, COUNT(*) AS num_failed_students FROM marks m JOIN students s ON s.student_id = m.student_id WHERE s.standard = '8th' AND m.grade = 'F' GROUP BY m.subject_name ORDER BY num_failed_students DESC LIMIT 1",
     'SQLResult': "Result of the SQL query",
     'Answer': qu10},

    {'Question': "Find out the number of male and female students in each class",
     'SQLQuery': "SELECT standard, gender, COUNT(*) AS count FROM students GROUP BY standard, gender",
     'SQLResult': "Result of the SQL query",
     'Answer': qu11},

    {'Question': "Determine the subjects taught by each teacher",
     'SQLQuery': "SELECT teacher_name, subject_taught FROM teachers",
     'SQLResult': "Result of the SQL query",
     'Answer': qu12},

    {'Question': "Calculate the average marks obtained by students in each subject for class 7th",
     'SQLQuery': "SELECT subject_name, ROUND(AVG(marks_obtained), 2) AS average_marks FROM marks JOIN students ON marks.student_id = students.student_id WHERE students.standard = '7th' GROUP BY subject_name",
     'SQLResult': "Result of the SQL query",
    'Answer': qu13},

    {'Question': "Find out the total number of students standardwise who attend coaching classes",
     'SQLQuery': "SELECT standard, COUNT(*) AS total_students, SUM(CASE WHEN coaching_class = 'Yes' THEN 1 ELSE 0 END) AS num_students_attending_coaching FROM students GROUP BY standard",
     'SQLResult': "Result of the SQL query",
     'Answer': qu14},

    {'Question': "List the names of students who scored the highest marks in each subject of 8th std",
     'SQLQuery': "WITH SubjectTopper AS ( SELECT m.subject_name, s.student_id, s.student_name, m.marks_obtained AS highest_marks, ROW_NUMBER() OVER (PARTITION BY m.subject_name ORDER BY m.marks_obtained DESC) AS rr FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '8th') SELECT subject_name, student_id, student_name, highest_marks FROM SubjectTopper WHERE rr = 1;",
     'SQLResult': "Result of the SQL query",
     'Answer': qu15},

    {'Question': "Determine the number of students who failed in more than one subjects.",
     'SQLQuery': "SELECT COUNT(*) AS num_students_failed_in_more_than_two_subjects FROM ( SELECT m.student_id FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '6th' AND m.grade = 'F' GROUP BY m.student_id HAVING COUNT(*) > 1) AS failed_students",
     'SQLResult': "Result of the SQL query",
     'Answer': qu16},

    {'Question': "Calculate the overall pass percentage of students in each class",
     'SQLQuery': "SELECT standard, ROUND((COUNT(CASE WHEN grade != 'F' THEN 1 END) / COUNT(*) * 100), 2) AS pass_percentage FROM students JOIN marks ON students.student_id = marks.student_id GROUP BY standard ORDER BY standard",
     'SQLResult': "Result of the SQL query",
     'Answer': qu17},

    {'Question': "Retrieve the name of the student who got an 'A' grade in computer for class 6th",
     'SQLQuery': "SELECT student_name FROM students JOIN marks USING (student_id) WHERE grade = 'A' AND subject_name = 'Computer' AND standard = '6th'",
     'SQLResult': "Result of the SQL query",
     'Answer': qu18},

    {'Question': "Display the results of students SI01 and SI07 in a comparative form",
     'SQLQuery': "SELECT subject_name, MAX(CASE WHEN student_id = 'SI01' THEN marks_obtained END) AS SI01_marks, MAX(CASE WHEN student_id = 'SI07' THEN marks_obtained END) AS SI07_marks, MAX(CASE WHEN student_id = 'SI01' THEN grade END) AS SI01_grade, MAX(CASE WHEN student_id = 'SI07' THEN grade END) AS SI07_grade FROM marks WHERE student_id IN ('SI01', 'SI07') GROUP BY subject_name",
     'SQLResult': "Result of the SQL query",
     'Answer': qu19},

    {'Question': "Find the first three ranker of class 7th",
     'SQLQuery': "SELECT m.student_id, s.student_name, '7th' AS class, SUM(m.marks_obtained) AS total_marks FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '7th' GROUP BY m.student_id, s.student_name ORDER BY total_marks DESC LIMIT 3",
     'SQLResult': "Result of the SQL query",
     'Answer': qu20},
]

#### Creating Semantic Similarity Based example selector


1) create embedding on the few_shots
2) Store the embeddings in Chroma DB
3) Retrieve the the top most Semantically close example from the vector store

In [168]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

e = embeddings.embed_query("Find out which subject Sohel teaches")

In [169]:
e[:5]

[0.09965822845697403,
 0.014540574513375759,
 -0.08289855718612671,
 -0.006000849884003401,
 0.11131759732961655]

In [174]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["Find out which subject Ankita Mundada teaches SELECT subject_taught FROM teachers WHERE teacher_name = 'Ankita Mundada' Result of the SQL query Maths",
 "Percentage of student id SI07 SELECT s.student_id, s.student_name, ROUND((SUM(m.marks_obtained) / COUNT(m.mark_id)), 2) AS percentage FROM students s JOIN marks m USING(student_id) WHERE s.student_id = 'SI07' Result of the SQL query Kabir Thapar, 49.20",
 "Find out who got the highest marks in Maths among 8th standard students and how many marks they scored. SELECT s.student_name, m.marks_obtained AS highest_marks FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.standard = '8th' AND m.subject_name = 'Maths' ORDER BY m.marks_obtained DESC LIMIT 1 Result of the SQL query Nikhil Pandit scored 100 marks in Maths",
 "Find the average marks in English of 7th standard students? SELECT ROUND(AVG(marks_obtained), 2) AS average_marks FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '7th' AND m.

In [175]:
to_vectorize[0]

"Find out which subject Ankita Mundada teaches SELECT subject_taught FROM teachers WHERE teacher_name = 'Ankita Mundada' Result of the SQL query Maths"

In [176]:
# create vector database
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [178]:
# to show similar examples
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "List the names of students who scored the highest marks in each subject of 8th std"})

[{'Answer': 'Computer: Omkar Khair, 97; English: Omkar Khair, 93; History: Ranjan Kumar, 96; Maths: Nikhil Pandit, 100; Science: Abhi Karve, 94',
  'Question': 'List the names of students who scored the highest marks in each subject of 8th std',
  'SQLQuery': "WITH SubjectTopper AS ( SELECT m.subject_name, s.student_id, s.student_name, m.marks_obtained AS highest_marks, ROW_NUMBER() OVER (PARTITION BY m.subject_name ORDER BY m.marks_obtained DESC) AS rr FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '8th') SELECT subject_name, student_id, student_name, highest_marks FROM SubjectTopper WHERE rr = 1;",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'Nikhil Pandit scored 100 marks in Maths',
  'Question': 'Find out who got the highest marks in Maths among 8th standard students and how many marks they scored.',
  'SQLQuery': "SELECT s.student_name, m.marks_obtained AS highest_marks FROM students s JOIN marks m ON s.student_id = m.student_id WHERE s.s

In [183]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [181]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [184]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [186]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [188]:
# Create new chain

new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [192]:
new_qu14 = new_chain.run("Find out the total number of students standard wise who attend coaching classes")
new_qu14



> Entering new SQLDatabaseChain chain...
Find out the total number of students standard wise who attend coaching classes
SQLQuery:SELECT standard, COUNT(*) FROM students WHERE coaching_class = 'Yes' GROUP BY standard
SQLResult: [('8th', 13), ('7th', 11), ('6th', 14)]
Answer:38
> Finished chain.


'38'

In [200]:
new_qu20 = new_chain("Determine the top three rankers in terms of total marks among students of class 8th")
new_qu20



> Entering new SQLDatabaseChain chain...
Determine the top three rankers in terms of total marks among students of class 8th
SQLQuery:SELECT m.student_id, s.student_name, '8th' AS class, SUM(m.marks_obtained) AS total_marks FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '8th' GROUP BY m.student_id, s.student_name ORDER BY total_marks DESC LIMIT 3
SQLResult: [('EI20', 'Abhi Karve', '8th', Decimal('434')), ('EI18', 'Kishor Kumar', '8th', Decimal('393')), ('EI01', 'Piyusha Kadam', '8th', Decimal('389'))]
Answer:Abhi Karve, Kishor Kumar, Piyusha Kadam
> Finished chain.


In [202]:
# askes question in different way
new_chain.run("Determine the top three rankers in terms of total marks among students of class 8th")



> Entering new SQLDatabaseChain chain...
Determine the top three rankers in terms of total marks among students of class 8th
SQLQuery:SELECT m.student_id, s.student_name, '8th' AS class, SUM(m.marks_obtained) AS total_marks FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '8th' GROUP BY m.student_id, s.student_name ORDER BY total_marks DESC LIMIT 3
SQLResult: [('EI20', 'Abhi Karve', '8th', Decimal('434')), ('EI18', 'Kishor Kumar', '8th', Decimal('393')), ('EI01', 'Piyusha Kadam', '8th', Decimal('389'))]
Answer:Abhi Karve, Kishor Kumar, Piyusha Kadam
> Finished chain.


'Abhi Karve, Kishor Kumar, Piyusha Kadam'

In [209]:
# asked in different way
new_chain.run("Determine first rankers in terms of total marks among students of class 6th")



> Entering new SQLDatabaseChain chain...
Determine first rankers in terms of total marks among students of class 6th
SQLQuery:SELECT m.student_id, s.student_name, '6th' AS class, SUM(m.marks_obtained) AS total_marks FROM marks m JOIN students s ON m.student_id = s.student_id WHERE s.standard = '6th' GROUP BY m.student_id, s.student_name ORDER BY total_marks DESC LIMIT 1
SQLResult: [('SI21', 'Seema Kadam', '6th', Decimal('398'))]
Answer:Seema Kadam
> Finished chain.


'Seema Kadam'

In [211]:
new_qu19 = new_chain.run("display the results of students SE01 and SE07 in a comparative form")
new_qu19



> Entering new SQLDatabaseChain chain...
display the results of students SE01 and SE07 in a comparative form
SQLQuery:SELECT subject_name, MAX(CASE WHEN student_id = 'SE01' THEN marks_obtained END) AS SE01_marks, MAX(CASE WHEN student_id = 'SE07' THEN marks_obtained END) AS SE07_marks, MAX(CASE WHEN student_id = 'SE01' THEN grade END) AS SE01_grade, MAX(CASE WHEN student_id = 'SE07' THEN grade END) AS SE07_grade FROM marks WHERE student_id IN ('SE01', 'SE07') GROUP BY subject_name
SQLResult: [('English', 53, 95, 'D', 'A'), ('Computer', 72, 96, 'B', 'A'), ('History', 38, 44, 'F', 'E'), ('Maths', 86, 91, 'A', 'A'), ('Science', 99, 49, 'A', 'E')]
Answer:English, 53, 95, D, A
> Finished chain.


'English, 53, 95, D, A'

In [218]:
# rewrit question with replacing average to mean3

new_qu13 = new_chain.run("Find the mean marks scored by 7th-grade students in each subject")
new_qu13



> Entering new SQLDatabaseChain chain...
Find the mean marks scored by 7th-grade students in each subject
SQLQuery:SELECT subject_name, AVG(marks_obtained) AS mean_marks FROM marks JOIN students ON marks.student_id = students.student_id WHERE students.standard = '7th' GROUP BY subject_name
SQLResult: [('English', Decimal('69.3200')), ('Computer', Decimal('64.6000')), ('History', Decimal('58.2800')), ('Maths', Decimal('63.8400')), ('Science', Decimal('68.8000'))]
Answer:English, 69.32
> Finished chain.


'English, 69.32'

In [219]:
new_qu2 = new_chain.run("what is percentage of student id SI07")
new_qu2



> Entering new SQLDatabaseChain chain...
what is percentage of student id SI07
SQLQuery:SELECT s.student_id, s.student_name, ROUND((SUM(m.marks_obtained) / COUNT(m.mark_id)), 2) AS percentage FROM students s JOIN marks m USING(student_id) WHERE s.student_id = 'SI07'
SQLResult: [('SI07', 'Kabir Thapar', Decimal('49.20'))]
Answer:49.20
> Finished chain.


'49.20'

In [220]:
# give student name instead of student id
new_qu2 = new_chain.run("what is percentage of student Kabir Thapar")



> Entering new SQLDatabaseChain chain...
what is percentage of student Kabir Thapar
SQLQuery:SELECT s.student_id, s.student_name, ROUND((SUM(m.marks_obtained) / COUNT(m.mark_id)), 2) AS percentage FROM students s JOIN marks m USING(student_id) WHERE s.student_id = 'SI07'
SQLResult: [('SI07', 'Kabir Thapar', Decimal('49.20'))]
Answer:49.20
> Finished chain.


In [223]:
new_qu2 = new_chain.run("Calculate the average marks of student id SI07")
new_qu2



> Entering new SQLDatabaseChain chain...
Calculate the average marks of student id SI07
SQLQuery:SELECT ROUND(AVG(marks_obtained), 2) AS average_marks FROM marks WHERE student_id = 'SI07'
SQLResult: [(Decimal('49.20'),)]
Answer:49.20
> Finished chain.


'49.20'